In [169]:
import numpy as np
import torch
from torch.nn import functional as F
from tqdm import tqdm
import onnx
import onnxruntime
from onnx import numpy_helper
import os
import time

In [170]:

def extract_onnx_weight(onnx_path):
    model = onnx.load(onnx_path)

    weights, names = [], []
    for t in model.graph.initializer:
        weights.append(numpy_helper.to_array(t))
        names.append(t.name)
        
    onnx_weight = dict()
    for name, weight in zip(names, weights):
        onnx_weight[name] = weight
    #return onnx_weight
    print(onnx_weight)
    
#extract_onnx_weight("deit_quant_0519.onnx")

In [171]:
import onnx
import onnxruntime
import collections

# 加载ONNX模型
onnx_model_path = "deit_quant_0519.onnx"  # 替换为你的ONNX模型文件路径
model = onnx.load(onnx_model_path)

# 通过名称查找目标节点
target_node_name = "Conv_3_quant"
found = False
for node in model.graph.node:
    if node.name == target_node_name:
        found = True
        break

if found:
    # 提取节点的输入和输出名称
    input_names = node.input
    output_names = node.output

    print(f"输入名称: {input_names}")
    print(f"输出名称: {output_names}")

    # 如果需要，你也可以提取节点的属性
    attributes = node.attribute
    for attr in attributes:
        print(f"属性名称: {attr.name}, 属性值: {attr.s}")

else:
    print(f"未在模型中找到节点 '{target_node_name}'。")



输入名称: ['x_quantized', 'x_scale', 'x_zero_point', 'patch_embed.proj.weight_quantized', 'patch_embed.proj.weight_scale', 'patch_embed.proj.weight_zero_point', '156_scale', '156_zero_point', 'patch_embed.proj.bias_quantized']
输出名称: ['156_quantized']
属性名称: dilations, 属性值: b''
属性名称: group, 属性值: b''
属性名称: kernel_shape, 属性值: b''
属性名称: pads, 属性值: b''
属性名称: strides, 属性值: b''


In [175]:
import onnxruntime
#deit_quant_0519
import numpy as np

# 加载ONNX模型
onnx_model_path = "deit_quant_0519.onnx"  # 替换为你的ONNX模型文件路径
session = onnxruntime.InferenceSession(onnx_model_path)

# 检查模型的输入要求
input_names = [input.name for input in session.get_inputs()]
print("检查模型的输入:", input_names)

# 确保提供名为 "x" 的输入
if "x" in input_names:
    # 获取名为 "x" 的输入的形状
    input_shape = session.get_inputs()[input_names.index("x")].shape

    # 创建一个名为 "x" 的输入张量
    x_data = np.random.randn(*input_shape).astype(np.float32)

    # 运行模型
    output = session.run([], {'x': x_data})

    # 输出结果
    print("模型输出:", output)
else:
    print("模型需要名为 'x' 的输入，但未找到。")

检查模型的输入: ['x']
模型需要名为 'x' 的输入，但未找到。


In [173]:
def extract_onnx_activate(model_path):
    
    model = onnx.load(model_path)
    #打印信息
    print("==============node信息")
    # print(helper.printable_graph(model.graph))
    # print(model)

    conv_weight = model.graph.initializer[1]
    conv_bias = model.graph.initializer[1]
    print(conv_weight)
    #initializer里有dims这个属性是可以通过打印model看到的
    #打印输出的是1，dims在onnx-ml.proto文件中是repeated类型的，即数组类型，所以要用索引去取！
    print(conv_weight.dims[0])
    #取node节点的第一个元素
    print("取node节点的第一个元素")
    print("xxxxx",model.graph.node[1])

    # 数据是以protobuf的格式存储的，因此当中的数值会以bytes的类型保存，通过np.frombuffer方法还原成类型为float32的ndarray
    print(f"===================={conv_weight.name}==========================")
    print(conv_weight.name, np.frombuffer(conv_weight.raw_data, dtype=np.int8))

    print(f"===================={conv_bias.name}==========================")
    print(conv_bias.name, np.frombuffer(conv_bias.raw_data, dtype=np.int8))

    
extract_onnx_activate("deit_quant_0519.onnx")

==============node信息
dims: 1
dims: 197
dims: 192
data_type: 1
name: "pos_embed"
raw_data: "\355j\261<\326D\303\274\344V8\277\365&\226=\211A\256>\373\312~\277$\237\t=\263\024\322>?\352\265\276\363~\214\277T\262\261>\371\317\217\275\010WL\276\270\363\327>\275\177Y\276\2442r\276$q\035\277\363\355\240\276\226\205+\276\266\340\'\276w\334\201?K4\334@\301f\234=(5&\277\200[6?h\372\341\276\325+\331\274>\035U>\322\020\014>\201\214\356\276W\334\224\276\247*U>\'\256\314>\257\3456>\206\214\010\277\272>\033\277\024V\210=?\031\325>\266\027G\277\031S|\274\321\023\032\277YV0>\371E\354\276\373\214\016?\032,9\276\233\014=\276\213\304 \276\032k\211>c/\261\276J\234\001>@\330\344\276l\360Y?\253\305i<\037\035\261<$\014\r\277i\332\023\277\243\331X=78\224=\304O\022?\367\007\030>\001\231 \277\240R\264\275\310\301B\274\320\230\254\276#:\344\273@\265\325>\t\030\033?\265\357V=v\002\345=\213\340\266\276\214 \\>\305\324\203>3Z\322\276\346cM?\022\226\235\276;\274\367\275=E\032?T\334\002\277|\261-;\034\376\375\275=b\2

In [174]:
import os
import onnx
import copy
import numpy as np
import logging
import onnxruntime
from collections import OrderedDict
from onnx import shape_inference
logging.basicConfig(level=logging.INFO)
from onnx import shape_inference, TensorProto, version_converter, numpy_helper
logger = logging.getLogger("[ONNXOPTIMIZER]")


def test_model_by_onnxruntime(model):
    logger.info("Test model by onnxruntime")

    input_shape = model.graph.input[0].type.tensor_type.shape.dim

    image_shape = [x.dim_value for x in input_shape]
    image_shape_new = []
    for x in image_shape:
        if x == 0:
            image_shape_new.append(1)
        else:
            image_shape_new.append(x)
    image_shape = image_shape_new
    img_array = np.array(np.random.random(image_shape), dtype = np.float32)
    img = img_array
    for node in model.graph.node:
        for output in node.output:
            model.graph.output.extend([onnx.ValueInfoProto(name=output)])
    ort_session = onnxruntime.InferenceSession(model.SerializeToString())
    ort_inputs = {}
    for i, input_ele in enumerate(ort_session.get_inputs()):
        ort_inputs[input_ele.name] = img

    outputs = [x.name for x in ort_session.get_outputs()]
    ort_outs = ort_session.run(outputs, ort_inputs)
    ort_outs = OrderedDict(zip(outputs, ort_outs))
    
    logger.info("Test model by onnxruntime success")
    #print(ort_outs)
    '''del model.graph.output[:]
    model.graph.output.extend(ori_output)
    return ort_outs'''

onnx_model = onnx.load("deit_quant_0519.onnx")
ort_outs = test_model_by_onnxruntime(onnx_model)


INFO:[ONNXOPTIMIZER]:Test model by onnxruntime
INFO:[ONNXOPTIMIZER]:Test model by onnxruntime success
